In [17]:
import flightradar24
import pandas as pd

fr = flightradar24.Api()
airports = fr.get_airports()

In [18]:
len(airports["rows"])

4297

In [19]:
airports

{'version': '1565157600',
 'rows': [{'name': '\tPraslin Island Airport',
   'iata': 'PRI',
   'icao': 'FSPP',
   'lat': -4.31929,
   'lon': 55.69141,
   'country': 'Seychelles',
   'alt': 10},
  {'name': '\r\nLong Seridan Airport',
   'iata': 'ODN',
   'icao': 'WBGI',
   'lat': 3.976111,
   'lon': 115.063301,
   'country': 'Indonesia',
   'alt': 607},
  {'name': 'A Coruna Airport',
   'iata': 'LCG',
   'icao': 'LECO',
   'lat': 43.302059,
   'lon': -8.37725,
   'country': 'Spain',
   'alt': 326},
  {'name': 'Aachen Merzbruck Airport',
   'iata': 'AAH',
   'icao': 'EDKA',
   'lat': 50.823051,
   'lon': 6.186111,
   'country': 'Germany',
   'alt': 623},
  {'name': 'Aalborg Airport',
   'iata': 'AAL',
   'icao': 'EKYT',
   'lat': 57.092781,
   'lon': 9.849164,
   'country': 'Denmark',
   'alt': 10},
  {'name': 'Aarhus Airport',
   'iata': 'AAR',
   'icao': 'EKAH',
   'lat': 56.300011,
   'lon': 10.619,
   'country': 'Denmark',
   'alt': 82},
  {'name': 'Aarhus Sea Airport',
   'iata': 'QE

In [20]:
len(airports["rows"])

4297

Creating a dataframe that keeps location of airports with airport iata name

In [80]:
iata = []
lon = []
lat = []
for ap in airports["rows"]:
    iata.append(ap["iata"])
    lon.append(ap["lon"])
    lat.append(ap["lat"])

In [81]:
df = pd.DataFrame()
df["iata"] = iata
df["lon"] = lon
df["lat"] = lat
df.head()

,iata,lon,lat
0,PRI,55.691410,-4.319290
1,ODN,115.063301,3.976111
2,LCG,-8.377250,43.302059
3,AAH,6.186111,50.823051
4,AAL,9.849164,57.092781


Following code segment plots a map with given origin and destination airports

In [82]:
import folium
import numpy as np
import pandas as pd
from collections import namedtuple

def get_arrows(locations, color='blue', size=6, n_arrows=3):
    Point = namedtuple('Point', field_names=['lat', 'lon'])
    
    # creating point from our Point named tuple
    p1 = Point(locations[0][0], locations[0][1])
    p2 = Point(locations[1][0], locations[1][1])
    
    # getting the rotation needed for our marker.  
    # Subtracting 90 to account for the marker's orientation
    # of due East(get_bearing returns North)
    rotation = get_bearing(p1, p2) - 90
    
    # get an evenly space list of lats and lons for our arrows
    # note that I'm discarding the first and last for aesthetics
    # as I'm using markers to denote the start and end
    arrow_lats = np.linspace(p1.lat, p2.lat, n_arrows + 2)[1:n_arrows+1]
    arrow_lons = np.linspace(p1.lon, p2.lon, n_arrows + 2)[1:n_arrows+1]
    
    arrows = []
    
    #creating each "arrow" and appending them to our arrows list
    for points in zip(arrow_lats, arrow_lons):
        arrows.append(folium.RegularPolygonMarker(location=points, 
                      fill_color=color, number_of_sides=3, 
                      radius=size, rotation=rotation))
    return arrows
def get_middle(p1,p2):
    return [(p1[0]+p2[0])/2, (p1[1]+p2[1])/2]

def get_bearing(p1, p2):    
    long_diff = np.radians(p2.lon - p1.lon)
    
    lat1 = np.radians(p1.lat)
    lat2 = np.radians(p2.lat)
    
    x = np.sin(long_diff) * np.cos(lat2)
    y = (np.cos(lat1) * np.sin(lat2) 
        - (np.sin(lat1) * np.cos(lat2) 
        * np.cos(long_diff)))
    bearing = np.degrees(np.arctan2(x, y))
    
    # adjusting for compass bearing
    if bearing < 0:
        return bearing + 360
    return bearing

# using omaha coordinates 
center_lat = 42.211898
center_lon = -38.394936

In [83]:
df.loc[df["iata"] == "JFK"]

,iata,lon,lat
2647,JFK,-73.7789,40.639751


In [84]:
def get_cor(airport):
    p1 = []
    p1.append(list(df[df["iata"] == airport]["lat"])[0])
    p1.append(list(df[df["iata"] == airport]["lon"])[0])
    return p1

In [85]:
p1 = get_cor("LAX")
p1

[33.94252, -118.406998]

In [86]:
p2 = get_cor("JFK")
p2

[40.639751, -73.7789]

In [87]:
# generating a couple of random latlongs in the omaha area
lats = np.random.uniform(low=center_lat - .25, high=center_lat + .25, size=(2,))
lons = np.random.uniform(low=center_lon - .25, high=center_lon + .25, size=(2,))

some_map = folium.Map(location=[center_lat, center_lon], zoom_start=3)
folium.Marker(location=p1,  icon=folium.Icon(color='green')).add_to(some_map)
icon_url = "https://image.flaticon.com/icons/svg/1085/1085919.svg"
icon = folium.features.CustomIcon(icon_url,icon_size=(50, 50))
folium.Marker(location=get_middle(p1,p2),icon=icon).add_to(some_map)
folium.Marker(location=p2, icon=folium.Icon(color='blue')).add_to(some_map)

folium.PolyLine(locations=[p1, p2], color='red').add_to(some_map)

icon_url = "https://www.flaticon.com/free-icon/airplane_579268#term=plane&page=1&position=18"
icon = folium.features.CustomIcon(icon_url,icon_size=(25, 25))

"""
arrows = get_arrows(locations=[p1, p2], n_arrows=1)
for arrow in arrows:
    arrow.add_to(some_map)
    
"""

some_map

In [88]:
flight_id = 'TK1' # Turkish Airlines' Istanbul - New York flight
fr = flightradar24.Api()
flight = fr.get_flight(flight_id)

In [ ]:
airline = 'THY' # Turkish Airlines
fr = flightradar24.Api()
flights = fr.get_flights(airline)